In [21]:
import pandas as pd

# Load
df = pd.read_csv("C:/Users/hp/Downloads/SuperStore_Sales_Dataset.csv")

# 1. Normalize column names (lowercase, underscores)
df.columns = [c.strip().replace(' ', '_').lower() for c in df.columns]

# 2. Robust date parsing: try expected format, fallback
def safe_parse(s):
    s = str(s).strip()
    try:
        return pd.to_datetime(s, format="%d-%b-%y")  # e.g., "01-Jan-19"
    except Exception:
        return pd.to_datetime(s, dayfirst=True, errors='coerce')

df['order_date'] = safe_parse(df.get('date'))
df['ship_date'] = safe_parse(df.get('ship_date'))

# 3. Shipping delay
df['shipping_days'] = (df['ship_date'] - df['order_date']).dt.days

# 4. Impute shipping_days with median (with fallback)
median_delay = df['shipping_days'].median()
if pd.isna(median_delay):
    median_delay = 4
df['shipping_days'] = df['shipping_days'].fillna(median_delay)

# 5. Reconstruct missing one of the dates if possible
mask_ship_missing = df['ship_date'].isna() & df['order_date'].notna()
df.loc[mask_ship_missing, 'ship_date'] = df.loc[mask_ship_missing, 'order_date'] + pd.to_timedelta(df.loc[mask_ship_missing, 'shipping_days'], unit='d')

mask_order_missing = df['order_date'].isna() & df['ship_date'].notna()
df.loc[mask_order_missing, 'order_date'] = df.loc[mask_order_missing, 'ship_date'] - pd.to_timedelta(df.loc[mask_order_missing, 'shipping_days'], unit='d')

# 6. Return flag
df['return_flag'] = df.get('returns', 0).apply(lambda x: 'yes' if pd.notna(x) and x == 1 else 'no')

# 7. Trim and fill categorical/text columns
text_cols = ['ship_mode', 'category', 'sub-category', 'region', 'segment', 'state', 'city', 'product_name']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().replace({'nan': pd.NA})
        mode = df[col].mode()
        if not mode.empty:
            df[col] = df[col].fillna(mode[0])

# 8. Numeric enforcement + fill with median (fallback to 0)
for num in ['sales', 'profit', 'quantity', 'discount']:
    if num in df.columns:
        df[num] = pd.to_numeric(df[num], errors='coerce')
        med = df[num].median()
        df[num] = df[num].fillna(0 if pd.isna(med) else med)

# 9. Shipping validity flag
df['shipping_validity'] = 'normal'
df.loc[df['shipping_days'] < 0, 'shipping_validity'] = 'ship_before_order'
df.loc[df['shipping_days'] > 15, 'shipping_validity'] = 'long_delay'

# 10. Remove duplicates
df = df.drop_duplicates()

# 11. Audit timestamp
df['cleaned_timestamp'] = pd.Timestamp.now()

# 12. Save cleaned dataset
df.to_csv("superstore_sales_cleaned.csv", index=False)

# 13. Diagnostics
print("Final shape:", df.shape)
print("Missing per column:\n", df.isnull().mean().sort_values(ascending=False).head(10))
print("Return flag counts:\n", df['return_flag'].value_counts())
print("Shipping validity:\n", df['shipping_validity'].value_counts())


Final shape: (5901, 26)
Missing per column:
 ship_date            1.000000
order_date           1.000000
returns              0.951364
row_id+o6g3a1:r6     0.000000
sub-category         0.000000
shipping_validity    0.000000
return_flag          0.000000
shipping_days        0.000000
payment_mode         0.000000
profit               0.000000
dtype: float64
Return flag counts:
 return_flag
no     5614
yes     287
Name: count, dtype: int64
Shipping validity:
 shipping_validity
normal    5901
Name: count, dtype: int64
